In [105]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# -----------------------------
# CONFIGURATION
# -----------------------------
URL = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
DB_NAME = 'Movies.db'
TABLE_NAME = 'Top_50'
CSV_PATH = 'top_50_films.csv'
LOG_FILE = 'etl_log.txt'



# -----------------------------
# LOGGING FUNCTION
# -----------------------------
def log_progress(message):
    """Log the progress of ETL steps with timestamps."""
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(LOG_FILE, 'a') as f:
        f.write(f"{timestamp} - {message}\n")


# -----------------------------
# EXTRACT FUNCTION
# -----------------------------


def extract(url):
    """Extract top 50 movie data from the webpage."""
    html_page = requests.get(url).text
    data = BeautifulSoup(html_page, 'html.parser')

    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')

    df = pd.DataFrame(columns=[
        "Film", "Year", "IMDb's Top 250",
        "Rotten Tomatoes' Top 100", "Empire's Top 100",
        "AFI's Top 100", "BFI's Top 100"
    ])
    
    count = 0
    for row in rows:
        if count >= 50:  # 50 filmi aldıysak dur
            break

        col = row.find_all('td')
        if len(col) != 0:
            data_dict = {
                'Film': col[1].text.strip(),
                'Year': col[2].text.strip(),
                "Rotten Tomatoes' Top 100": col[3].text.strip(),
                "IMDb's Top 250": col[4].text.strip(),
                "Empire's Top 100": col[5].text.strip(),
                "AFI's Top 100": col[6].text.strip(),
                "BFI's Top 100": col[7].text.strip()
            }
            df = pd.concat([df, pd.DataFrame([data_dict])], ignore_index=True)
            count += 1

    log_progress("Extraction completed successfully.")
    return df


# -----------------------------
# TRANSFORM FUNCTION
# -----------------------------
def transform(df):
    for col in df.columns:
        if col != "Film":
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0).astype(int)  # NaN'ları 0 yapıyoruz
    log_progress("Transformation completed successfully.")
    return df

# -----------------------------
# LOAD FUNCTION
# -----------------------------
def load(df, csv_path, db_name, table_name):
    """Save the data into CSV and SQLite database."""
    df.to_csv(csv_path, index=False)
    log_progress(f"Data saved to CSV: {csv_path}")

    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    log_progress(f"Data loaded into database: {db_name} (table: {table_name})")


# -----------------------------
# MAIN ETL PIPELINE
# -----------------------------
def run_etl():
    log_progress("ETL Job Started")
    data = extract(URL)
    data = transform(data)
    load(data, CSV_PATH, DB_NAME, TABLE_NAME)
    log_progress("ETL Job Completed Successfully")
    return data


# Run the ETL process
if __name__ == "__main__":
    run_etl()


In [106]:
pd.read_csv('top_50_films.csv')

,Film,Year,IMDb's Top 250,Rotten Tomatoes' Top 100,Empire's Top 100,AFI's Top 100,BFI's Top 100
0,The Godfather,1972,2,17,3,2,21
1,Citizen Kane,1941,93,2,40,1,2
2,Casablanca,1942,42,8,33,3,84
3,"The Godfather, Part II",1974,4,99,14,32,31
4,Singin' in the Rain,1952,85,52,67,5,20
5,Psycho,1960,32,88,81,14,35
6,Rear Window,1954,49,47,72,48,53
7,Apocalypse Now,1979,55,0,29,30,14
8,2001: A Space Odyssey,1968,89,0,23,15,6
9,Seven Samurai,1954,20,49,65,0,17
